In [2]:
import dgl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor 
from torch.utils.data.dataloader import DataLoader
from dgl.data import CoraGraphDataset, CiteseerGraphDataset
import dgl.function as fn
from functools import namedtuple
from typing import Optional
from torch_geometric.typing import OptTensor
from torch_geometric.utils import scatter
import torch as th
from torch.nn import init
from dgl.utils import expand_as_pair
import scipy.sparse as sp
import itertools
from sklearn.metrics import roc_auc_score
import random

In [3]:
def Load_graph(data : str):
    if data == 'cora':
        dataset = CoraGraphDataset()
    elif data == 'citeseer':
        dataset = CiteseerGraphDataset()
    else:
        raise Exception('输入错误')
    return dataset

In [4]:
import torch
import torch.nn.functional as F

class PairNormLayer:
    def __init__(self, mode='PN-SI', scale=1):
        self.mode = mode
        self.scale = scale

    def __call__(self, x_feature):
        col_mean = x_feature.mean(dim=0)
        if self.mode == 'PN':
            x_feature = x_feature - col_mean
            row_norm_mean = (1e-6 + x_feature.pow(2).sum(dim=1).mean()).sqrt()
            x_feature = self.scale * x_feature / row_norm_mean

        elif self.mode == 'PN-SI':
            x_feature = x_feature - col_mean
            row_norm_individual = (1e-6 + x_feature.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x_feature = self.scale * x_feature / row_norm_individual

        elif self.mode == 'PN-SCS':
            row_norm_individual = (1e-6 + x_feature.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x_feature = self.scale * x_feature / row_norm_individual - col_mean

        return x_feature




In [5]:
class DropEdge(nn.Module):
    def __init__(self, p):
        super(DropEdge, self).__init__()
        self.p = p

    def forward(self, g):
        src, dst = g.edges()
        num_edges = g.num_edges()

        # 创建节点列表，记录每个节点的连接情况
        node_list = [set() for _ in range(g.num_nodes())]
        for s, d in zip(src, dst):
            node_list[s.item()].add(d.item())
            node_list[d.item()].add(s.item())

        # 根据概率 p 随机删除边
        mask = torch.rand(num_edges) > self.p
        # 保留未被删除的边
        src = src[mask]
        dst = dst[mask]

        # 检查删除边后是否导致节点成为孤立节点
        new_node_list = [set() for _ in range(g.num_nodes())]
        for s, d in zip(src, dst):
            new_node_list[s.item()].add(d.item())
            new_node_list[d.item()].add(s.item())
        for s, d in zip(src, dst):
            if len(new_node_list[s.item()]) == 0 or len(new_node_list[d.item()]) == 0:
                # 如果任一节点成为孤立节点，则保留这条边
                src = torch.cat([src, torch.tensor([s.item()])])
                dst = torch.cat([dst, torch.tensor([d.item()])])

        g = dgl.graph((src, dst))
        return g


In [6]:
class AddSelfLoop(nn.Module):
    def __init__(self):
        super(AddSelfLoop, self).__init__()

    def forward(self, g):
        num_nodes = g.num_nodes()
        src = torch.arange(num_nodes)
        dst = torch.arange(num_nodes)
        g.add_edges(src, dst)
        return g

In [7]:
class EdgeWeightNorm(nn.Module):
    def __init__(self, norm="both", eps=0.0):
        super(EdgeWeightNorm, self).__init__()
        self._norm = norm
        self._eps = eps

    def forward(self, graph, edge_weight):
        with graph.local_scope():
            if isinstance(graph, DGLBlock):
                graph = block_to_graph(graph)

            dev = graph.device
            dtype = edge_weight.dtype
            graph.srcdata["_src_out_w"] = th.ones(
                graph.number_of_src_nodes(), dtype=dtype, device=dev
            )
            graph.dstdata["_dst_in_w"] = th.ones(
                graph.number_of_dst_nodes(), dtype=dtype, device=dev
            )
            graph.edata["_edge_w"] = edge_weight

            if self._norm == "both":
                reversed_g = reverse(graph)
                reversed_g.edata["_edge_w"] = edge_weight
                reversed_g.update_all(
                    fn.copy_e("_edge_w", "m"), fn.sum("m", "out_weight")
                )
                degs = reversed_g.dstdata["out_weight"] + self._eps
                norm = th.pow(degs, -0.5)
                graph.srcdata["_src_out_w"] = norm

            if self._norm != "none":
                graph.update_all(
                    fn.copy_e("_edge_w", "m"), fn.sum("m", "in_weight")
                )
                degs = graph.dstdata["in_weight"] + self._eps
                if self._norm == "both":
                    norm = th.pow(degs, -0.5)
                else:
                    norm = 1.0 / degs
                graph.dstdata["_dst_in_w"] = norm

            graph.apply_edges(
                lambda e: {
                    "_norm_edge_weights": e.src["_src_out_w"]
                    * e.dst["_dst_in_w"]
                    * e.data["_edge_w"]
                }
            )
            return graph.edata["_norm_edge_weights"]


class GraphConv(nn.Module):
    def __init__(
        self,
        in_feats,
        out_feats,
        norm="both",
        weight=True,
        bias=True,
        activation=None,
        allow_zero_in_degree=False,
    ):
        super(GraphConv, self).__init__()
        
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._norm = norm
        self._allow_zero_in_degree = allow_zero_in_degree

        if weight:
            self.weight = nn.Parameter(th.Tensor(in_feats, out_feats))
        else:
            self.register_parameter("weight", None)

        if bias:
            self.bias = nn.Parameter(th.Tensor(out_feats))
        else:
            self.register_parameter("bias", None)

        self.reset_parameters()

        self._activation = activation

    def reset_parameters(self):
        if self.weight is not None:
            #初始化权重参数为均匀分布
            init.xavier_uniform_(self.weight)
        if self.bias is not None:
            #最开始的偏置项设置为0
            init.zeros_(self.bias)

    def set_allow_zero_in_degree(self, set_value):
        self._allow_zero_in_degree = set_value

    def forward(self, graph, feat, weight=None, edge_weight=None):
        with graph.local_scope():
            aggregate_fn = fn.copy_u("h", "m")
            if edge_weight is not None:
                assert edge_weight.shape[0] == graph.num_edges()
                graph.edata["_edge_weight"] = edge_weight
                aggregate_fn = fn.u_mul_e("h", "_edge_weight", "m")

            
            feat_src, feat_dst = expand_as_pair(feat, graph)
            if self._norm in ["left", "both"]:
                degs = graph.out_degrees().to(feat_src).clamp(min=1)
                if self._norm == "both":
                    norm = th.pow(degs, -0.5)
                else:
                    norm = 1.0 / degs
                shp = norm.shape + (1,) * (feat_src.dim() - 1)
                norm = th.reshape(norm, shp)
                feat_src = feat_src * norm

            weight = self.weight

            if self._in_feats > self._out_feats:
                
                if weight is not None:
                    feat_src = th.matmul(feat_src, weight)
                graph.srcdata["h"] = feat_src
                graph.update_all(aggregate_fn, fn.sum(msg="m", out="h"))
                rst = graph.dstdata["h"]
            else:
                graph.srcdata["h"] = feat_src
                graph.update_all(aggregate_fn, fn.sum(msg="m", out="h"))
                rst = graph.dstdata["h"]
                if weight is not None:
                    rst = th.matmul(rst, weight)

            if self._norm in ["right", "both"]:
                degs = graph.in_degrees().to(feat_dst).clamp(min=1)
                if self._norm == "both":
                    norm = th.pow(degs, -0.5)
                else:
                    norm = 1.0 / degs
                shp = norm.shape + (1,) * (feat_dst.dim() - 1)
                norm = th.reshape(norm, shp)
                rst = rst * norm

            if self.bias is not None:
                rst = rst + self.bias

            if self._activation is not None:
                rst = self._activation(rst)

            return rst

In [8]:
#节点分类部分的训练代码
def train_class(g, model, epochs, lr):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=3,threshold=0.0001, verbose=True, min_lr=1e-8)
    best_val_acc = 0
    best_test_acc = 0
    best_model = None

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(epochs):
        logits = model(g, features)
        pred = logits.argmax(1)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f})'.format(e, loss, val_acc, best_val_acc))
        if scheduler is not None:
            scheduler.step(val_acc)  
    return best_model

def test_class(model, g, test_mask):
    model.eval()
    logits = model(g, g.ndata['feat'])
    pred = logits.argmax(1)
    labels = g.ndata['label']
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
    print('Test Accuracy: {:.4f}'.format(test_acc.item()))



In [9]:
#定义一个GCN块和整体的GCN网络
class GCNBlock(nn.Module):
    def __init__(self, in_feats, out_feats, activation=None, pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GCNBlock, self).__init__()
        self.conv = GraphConv(in_feats, out_feats,activation= F.relu)
        self.dropedge = DropEdge(p=drop_edge)
        self.self_loops = AddSelfLoop()
        self.norm = PairNormLayer()
        self.activation = activation
        self.pair_norm = pair_norm
        self.add_self_loops = add_self_loops
        self.drop_edge = drop_edge

    def forward(self, g, in_feat):
        if self.drop_edge:
            g = self.dropedge(g)
            self.add_self_loops = True
        if self.add_self_loops:
            g = self.self_loops(g)
        h = self.conv(g, in_feat)
        if self.pair_norm:
            h = self.norm(h)
        if self.activation:
            h = self.activation(h)
        return h

class GCN(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers, activation, dropout=0., 
                 pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GCN, self).__init__()
        if dropout:
            self.dropout = nn.Dropout(dropout)
        else:
            self.dropout = 0
        self.layers = nn.ModuleList()
        self.layer1 = GCNBlock(in_feats, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge)
        self.layers = nn.ModuleList()
        for i in range(n_layers - 1):
            self.layers.append(GCNBlock(n_hidden, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge))
        self.layers.append(GCNBlock(n_hidden, n_classes, activation, pair_norm=pair_norm,add_self_loops=add_self_loops,drop_edge=drop_edge))

    def forward(self, g, features):
        h = self.layer1(g, features)
        for idx, layer in enumerate(self.layers):
            if idx > 0 and self.dropout:
                h = self.dropout(h)
            h = layer(g, h)
        if self.dropout:
            h = self.dropout(h)
        return h

In [73]:
#Cora节点分类,layer=1，drop_edge=0.15，一个示例
torch.manual_seed(16)
dataset = Load_graph('cora')
g = dataset[0]#.to('cuda')
best_model = GCN(g.ndata['feat'].shape[1], 512, dataset.num_classes, 
            n_layers=1,
            activation=F.relu, 
            dropout=0, 
            pair_norm=False, 
            add_self_loops=False, 
            drop_edge=0.15)
best_model_state_dict = train_class(g, best_model, epochs=100, lr=0.004) 
best_model.load_state_dict(best_model_state_dict)
test_class(best_model, g, g.ndata['test_mask'])

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.946, val acc: 0.198 (best 0.198)
In epoch 5, loss: 1.827, val acc: 0.630 (best 0.630)
In epoch 10, loss: 1.602, val acc: 0.728 (best 0.744)
In epoch 15, loss: 1.294, val acc: 0.760 (best 0.762)
Epoch 00019: reducing learning rate of group 0 to 2.8000e-03.
In epoch 20, loss: 0.973, val acc: 0.770 (best 0.770)
In epoch 25, loss: 0.721, val acc: 0.770 (best 0.776)
Epoch 00028: reducing learning rate of group 0 to 1.9600e-03.
In epoch 30, loss: 0.547, val acc: 0.780 (best 0.786)
Epoch 00033: reducing learning rate of group 0 to 1.3720e-03.
In epoch 35, loss: 0.443, val acc: 0.784 (best 0.788)
Epoch 00038: reducing learning rate of group 0 to 9.6040e-04.
In epoch 40, loss: 0.397, val acc: 0.778 (best 0.788)
Epoch 00042: reducing learning rate of group 0 to 6.7228e-04.
In epoch 45, loss: 0.350

以下是链路预测的部分代码

In [10]:
#计算一个点对中两点的特征的内积
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

In [11]:
#计算样本评分和标签的二分类交叉熵
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

In [12]:
#计算ruc
def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).detach().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).detach().numpy()
    return roc_auc_score(labels, scores)

In [12]:
#一个划分正负样本和训练、测试、验证集的示例
dataset = Load_graph('cora')
g = dataset[0]
u, v = g.edges()

index= np.arange(g.number_of_edges())
np.random.shuffle(index)
test_size = int(0.1 * len(index))
val_size = int(0.1 * len(index))
train_size = len(index) - test_size - val_size  
test_pos_u, test_pos_v = u[index[:test_size]], v[index[:test_size]]
val_pos_u, val_pos_v = u[index[test_size:test_size + val_size]], v[index[test_size:test_size + val_size]]
train_pos_u, train_pos_v = u[index[test_size + val_size:]], v[index[test_size + val_size:]]

adj = sp.coo_matrix((np.ones(len(u)),(u.numpy(),v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u,neg_v = np.where(adj_neg!=0)

neg_index = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_index[:test_size]], neg_v[neg_index[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_index[test_size + val_size:]], neg_v[neg_index[test_size + val_size:]]
val_neg_u, val_neg_v = neg_u[neg_index[test_size:test_size + val_size]], neg_v[neg_index[test_size:test_size + val_size]]
train_g = dgl.remove_edges(g, index[:test_size+val_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [37]:
#模型训练示例
torch.manual_seed(17)
model = GCN(train_g.ndata['feat'].shape[1],  n_hidden=600, n_classes=1433, n_layers=3, activation=F.relu, dropout=0., 
                 pair_norm=True, add_self_loops=False, drop_edge=0.05)

pred = DotPredictor()
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.0002) #0.0001
best_auc = 0.
epoch = 100
best_model_1 = None
for e in range(epoch):
    #用train_g中去掉了测试集和验证集边的图以及剩下的点和特征训练，进行二分类
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)

    val_pos_score = pred(val_pos_g, h)
    val_neg_score = pred(val_neg_g, h)
    val_auc = compute_auc(val_pos_score, val_neg_score)
    
    if best_auc < val_auc:
        best_auc = val_auc
        best_model_1 = model.state_dict()
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(val_auc)
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  val_auc, best_auc))
best_model_state_dict = best_model_1
model.load_state_dict(best_model_state_dict)
h = model(train_g, train_g.ndata['feat'])
test_pos_score = pred(test_pos_g, h)
test_neg_score = pred(test_neg_g, h)
test_auc = compute_auc(test_pos_score, test_neg_score)
print(test_auc)

In epoch 0, loss: 0.627, val_auc: 0.817, best_auc: 0.817
In epoch 5, loss: 0.598, val_auc: 0.937, best_auc: 0.937
In epoch 10, loss: 0.577, val_auc: 0.955, best_auc: 0.955
In epoch 15, loss: 0.562, val_auc: 0.961, best_auc: 0.961
In epoch 20, loss: 0.551, val_auc: 0.962, best_auc: 0.963
In epoch 25, loss: 0.545, val_auc: 0.966, best_auc: 0.966
In epoch 30, loss: 0.541, val_auc: 0.967, best_auc: 0.967
In epoch 35, loss: 0.538, val_auc: 0.968, best_auc: 0.968
In epoch 40, loss: 0.535, val_auc: 0.967, best_auc: 0.968
In epoch 45, loss: 0.534, val_auc: 0.969, best_auc: 0.970
In epoch 50, loss: 0.533, val_auc: 0.971, best_auc: 0.971
In epoch 55, loss: 0.532, val_auc: 0.971, best_auc: 0.971
In epoch 60, loss: 0.531, val_auc: 0.971, best_auc: 0.972
In epoch 65, loss: 0.530, val_auc: 0.971, best_auc: 0.972
In epoch 70, loss: 0.530, val_auc: 0.972, best_auc: 0.972
In epoch 75, loss: 0.529, val_auc: 0.973, best_auc: 0.973
In epoch 80, loss: 0.529, val_auc: 0.972, best_auc: 0.973
In epoch 85, los

In [ ]:
import scipy.sparse as sp
import itertools
from sklearn.metrics import roc_auc_score
dataset = Load_graph('citeseer')
g = dataset[0]
u, v = g.edges()

index= np.arange(g.number_of_edges())
np.random.shuffle(index)
test_size = int(0.05 * len(index))
val_size = int(0.05 * len(index))
train_size = len(index) - test_size - val_size  
test_pos_u, test_pos_v = u[index[:test_size]], v[index[:test_size]]
val_pos_u, val_pos_v = u[index[test_size:test_size + val_size]], v[index[test_size:test_size + val_size]]
train_pos_u, train_pos_v = u[index[test_size + val_size:]], v[index[test_size + val_size:]]

adj = sp.coo_matrix((np.ones(len(u)),(u.numpy(),v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u,neg_v = np.where(adj_neg!=0)

neg_index = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_index[:test_size]], neg_v[neg_index[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_index[test_size + val_size:]], neg_v[neg_index[test_size + val_size:]]
val_neg_u, val_neg_v = neg_u[neg_index[test_size:test_size + val_size]], neg_v[neg_index[test_size:test_size + val_size]]
train_g = dgl.remove_edges(g, index[:test_size+val_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

model = GCN(train_g.ndata['feat'].shape[1],  n_hidden=2000, n_classes=3703, n_layers=3, activation=F.relu, dropout=0., 
                 pair_norm=False, add_self_loops=True, drop_edge=0.01)

pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001) #0.0001
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
epoch = 300
best_model_1 = None
for e in range(epoch):
    #用train_g中去掉了测试集和验证集边的图以及剩下的点和特征训练，进行二分类
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)
    train_auc = compute_auc(train_pos_score, train_neg_score)

    val_pos_score = pred(val_pos_g, h)
    val_neg_score = pred(val_neg_g, h)
    val_auc = compute_auc(val_pos_score, val_neg_score)
    
    if best_auc < val_auc:
        best_auc = val_auc
        best_model_1 = model.state_dict()
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(val_auc)
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  val_auc, best_auc))
best_model_state_dict = best_model_1
model.load_state_dict(best_model_state_dict)
h = model(train_g, train_g.ndata['feat'])
test_pos_score = pred(test_pos_g, h)
test_neg_score = pred(test_neg_g, h)
test_auc = compute_auc(test_pos_score, test_neg_score)
print(test_auc)

In [19]:
import scipy.sparse as sp
import itertools
from sklearn.metrics import roc_auc_score
dataset = Load_graph('citeseer')
g = dataset[0]
u, v = g.edges()

index= np.arange(g.number_of_edges())
np.random.shuffle(index)
test_size = int(0.1 * len(index))
val_size = int(0.1 * len(index))
train_size = len(index) - test_size - val_size  
test_pos_u, test_pos_v = u[index[:test_size]], v[index[:test_size]]
val_pos_u, val_pos_v = u[index[test_size:test_size + val_size]], v[index[test_size:test_size + val_size]]
train_pos_u, train_pos_v = u[index[test_size + val_size:]], v[index[test_size + val_size:]]

adj = sp.coo_matrix((np.ones(len(u)),(u.numpy(),v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u,neg_v = np.where(adj_neg!=0)

neg_index = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_index[:test_size]], neg_v[neg_index[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_index[test_size + val_size:]], neg_v[neg_index[test_size + val_size:]]
val_neg_u, val_neg_v = neg_u[neg_index[test_size:test_size + val_size]], neg_v[neg_index[test_size:test_size + val_size]]
train_g = dgl.remove_edges(g, index[:test_size+val_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

model = GCN(train_g.ndata['feat'].shape[1],  n_hidden=2000, n_classes=3703, n_layers=4, activation=F.relu, dropout=0., 
                 pair_norm=True, add_self_loops=False, drop_edge=0.01)

pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.002) #0.0001
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
epoch = 300
best_model_1 = None
for e in range(epoch):
    #用train_g中去掉了测试集和验证集边的图以及剩下的点和特征训练，进行二分类
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)
    train_auc = compute_auc(train_pos_score, train_neg_score)

    val_pos_score = pred(val_pos_g, h)
    val_neg_score = pred(val_neg_g, h)
    val_auc = compute_auc(val_pos_score, val_neg_score)
    
    if best_auc < val_auc:
        best_auc = val_auc
        best_model_1 = model.state_dict()
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(val_auc)
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  val_auc, best_auc))
best_model_state_dict = best_model_1
model.load_state_dict(best_model_state_dict)
h = model(train_g, train_g.ndata['feat'])
test_pos_score = pred(test_pos_g, h)
test_neg_score = pred(test_neg_g, h)
test_auc = compute_auc(test_pos_score, test_neg_score)
print(test_auc)

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 0.693, val_auc: 0.519, best_auc: 0.519
In epoch 5, loss: 0.688, val_auc: 0.597, best_auc: 0.597
In epoch 10, loss: 0.687, val_auc: 0.592, best_auc: 0.600
In epoch 15, loss: 0.686, val_auc: 0.593, best_auc: 0.600
In epoch 20, loss: 0.669, val_auc: 0.612, best_auc: 0.612
In epoch 25, loss: 0.637, val_auc: 0.615, best_auc: 0.620
In epoch 30, loss: 0.619, val_auc: 0.718, best_auc: 0.718
In epoch 35, loss: 0.610, val_auc: 0.757, best_auc: 0.758
In epoch 40, loss: 0.599, val_auc: 0.693, best_auc: 0.758
In epoch 45, loss: 0.599, val_auc: 0.776, best_auc: 0.776
In epoch 50, loss: 0.594, val_auc: 0.761, best_auc: 0.776
In epoch 55, loss: 0.593, val_auc: 0.761, best_auc: 0.776
In epoch 60, loss: 0.590, val_auc: 0.756, best_auc: 0.776
In epoch 65, loss: 0.589, val_auc: 0.754, best_auc: 0.776
In epoch 

In [ ]:
import scipy.sparse as sp
import itertools
from sklearn.metrics import roc_auc_score
dataset = Load_graph('citeseer')
g = dataset[0]
u, v = g.edges()

index= np.arange(g.number_of_edges())
np.random.shuffle(index)
test_size = int(0.1 * len(index))
val_size = int(0.1 * len(index))
train_size = len(index) - test_size - val_size  
test_pos_u, test_pos_v = u[index[:test_size]], v[index[:test_size]]
val_pos_u, val_pos_v = u[index[test_size:test_size + val_size]], v[index[test_size:test_size + val_size]]
train_pos_u, train_pos_v = u[index[test_size + val_size:]], v[index[test_size + val_size:]]

adj = sp.coo_matrix((np.ones(len(u)),(u.numpy(),v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u,neg_v = np.where(adj_neg!=0)

neg_index = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_index[:test_size]], neg_v[neg_index[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_index[test_size + val_size:]], neg_v[neg_index[test_size + val_size:]]
val_neg_u, val_neg_v = neg_u[neg_index[test_size:test_size + val_size]], neg_v[neg_index[test_size:test_size + val_size]]
train_g = dgl.remove_edges(g, index[:test_size+val_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

model = GCN(train_g.ndata['feat'].shape[1],  n_hidden=500, n_classes=3703, n_layers=5, activation=F.relu, dropout=0., 
                 pair_norm=False, add_self_loops=True, drop_edge=0.01)

pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.0002) #0.0001
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
epoch = 500
best_model_1 = None
for e in range(epoch):
    #用train_g中去掉了测试集和验证集边的图以及剩下的点和特征训练，进行二分类
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)
    train_auc = compute_auc(train_pos_score, train_neg_score)

    val_pos_score = pred(val_pos_g, h)
    val_neg_score = pred(val_neg_g, h)
    val_auc = compute_auc(val_pos_score, val_neg_score)
    
    if best_auc < val_auc:
        best_auc = val_auc
        best_model_1 = model.state_dict()
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(val_auc)
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  val_auc, best_auc))
best_model_state_dict = best_model_1
model.load_state_dict(best_model_state_dict)
h = model(train_g, train_g.ndata['feat'])
test_pos_score = pred(test_pos_g, h)
test_neg_score = pred(test_neg_g, h)
test_auc = compute_auc(test_pos_score, test_neg_score)
print(test_auc)

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 0.693, val_auc: 0.512, best_auc: 0.512


In [61]:
import scipy.sparse as sp
import itertools
from sklearn.metrics import roc_auc_score
dataset = Load_graph('citeseer')
g = dataset[0]
u, v = g.edges()

index= np.arange(g.number_of_edges())
np.random.shuffle(index)
test_size = int(0.05 * len(index))
val_size = int(0.05 * len(index))
train_size = len(index) - test_size - val_size  
test_pos_u, test_pos_v = u[index[:test_size]], v[index[:test_size]]
val_pos_u, val_pos_v = u[index[test_size:test_size + val_size]], v[index[test_size:test_size + val_size]]
train_pos_u, train_pos_v = u[index[test_size + val_size:]], v[index[test_size + val_size:]]

adj = sp.coo_matrix((np.ones(len(u)),(u.numpy(),v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u,neg_v = np.where(adj_neg!=0)

neg_index = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_index[:test_size]], neg_v[neg_index[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_index[test_size + val_size:]], neg_v[neg_index[test_size + val_size:]]
val_neg_u, val_neg_v = neg_u[neg_index[test_size:test_size + val_size]], neg_v[neg_index[test_size:test_size + val_size]]
train_g = dgl.remove_edges(g, index[:test_size+val_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

model = GCN(train_g.ndata['feat'].shape[1],  n_hidden=1024, n_classes=3703, n_layers=2, activation=F.relu, dropout=0., 
                 pair_norm=False, add_self_loops=False, drop_edge=0.01)

pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.0002) #0.0001
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
epoch = 500
best_model_1 = None
for e in range(epoch):
    #用train_g中去掉了测试集和验证集边的图以及剩下的点和特征训练，进行二分类
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)
    train_auc = compute_auc(train_pos_score, train_neg_score)

    val_pos_score = pred(val_pos_g, h)
    val_neg_score = pred(val_neg_g, h)
    val_auc = compute_auc(val_pos_score, val_neg_score)
    
    if best_auc < val_auc:
        best_auc = val_auc
        best_model_1 = model.state_dict()
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step(val_auc)
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  val_auc, best_auc))
best_model_state_dict = best_model_1
model.load_state_dict(best_model_state_dict)
h = model(train_g, train_g.ndata['feat'])
test_pos_score = pred(test_pos_g, h)
test_neg_score = pred(test_neg_g, h)
test_auc = compute_auc(test_pos_score, test_neg_score)
print(test_auc)

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 0.693, val_auc: 0.627, best_auc: 0.627
In epoch 5, loss: 0.690, val_auc: 0.609, best_auc: 0.647
In epoch 10, loss: 0.683, val_auc: 0.604, best_auc: 0.647
In epoch 15, loss: 0.674, val_auc: 0.614, best_auc: 0.647
In epoch 20, loss: 0.656, val_auc: 0.624, best_auc: 0.647
In epoch 25, loss: 0.629, val_auc: 0.653, best_auc: 0.653
In epoch 30, loss: 0.612, val_auc: 0.771, best_auc: 0.771
In epoch 35, loss: 0.605, val_auc: 0.786, best_auc: 0.786
In epoch 40, loss: 0.600, val_auc: 0.785, best_auc: 0.787
In epoch 45, loss: 0.597, val_auc: 0.790, best_auc: 0.792
In epoch 50, loss: 0.593, val_auc: 0.791, best_auc: 0.792
In epoch 55, loss: 0.589, val_auc: 0.791, best_auc: 0.793
In epoch 60, loss: 0.584, val_auc: 0.799, best_auc: 0.799
In epoch 65, loss: 0.578, val_auc: 0.811, best_auc: 0.811
In epoch 